In [ ]:
!pip install -q tensorflow==2.1

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
print("Трейн:", x_train.shape, y_train.shape)
print("Тест:", x_test.shape, y_test.shape)

Трейн: (50000, 32, 32, 3) (50000, 1)
Тест: (10000, 32, 32, 3) (10000, 1)


In [ ]:
NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

In [ ]:

cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_train))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid('off')
        ax.axis('off')
        ax.imshow(x_train[random_index, :])
        ax.set_title(cifar10_classes[y_train[random_index, 0]])
plt.show()

In [ ]:
# нормализуем входы
x_train2 = x_train.astype('float32') / 255 - 0.5
x_test2 = x_test.astype('float32') / 255 - 0.5

# конвертируем метки в np.array (?, NUM_CLASSES)
y_train2 = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test2 = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop

In [ ]:
def make_default_model():
    model = Sequential()
    model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(32, 32, 3)))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Flatten())
    model.add(L.Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(L.Dense(10, activation='softmax'))
    
    
    return model

In [ ]:
K.clear_session()
model = make_default_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0

In [ ]:
def train_model(make_model_func=make_default_model, optimizer="sgd"):
  BATCH_SIZE = 512
  EPOCHS = 100

  K.clear_session()
  model = make_model_func()

  model.compile(
      loss='categorical_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy']
  )

  model.fit(
      x_train2, y_train2,  # нормализованные данные
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      validation_data=(x_test2, y_test2),
      shuffle=False
  )
  
  return model

In [ ]:
# учим дефолтную архитектуру
train_model()

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
 2560/50000 [>.............................] - ETA: 14s - loss: 2.3079 - accuracy: 0.1246

KeyboardInterrupt: ignored

In [ ]:
def make_sigmoid_model():
    model = Sequential()
    model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(32, 32, 3)))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Flatten())
    model.add(L.Dense(256, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(L.Dense(10, activation='softmax'))
    return model

    

In [ ]:
    K.clear_session()
    model = make_sigmoid_model()
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0

In [ ]:
def train_model(make_model_func=make_sigmoid_model, optimizer="adam"):
  BATCH_SIZE = 512
  EPOCHS = 10

  K.clear_session()
  model = make_model_func()

  model.compile(
      loss='categorical_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy']
  )

  model.fit(
      x_train2, y_train2,  # нормализованные данные
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      validation_data=(x_test2, y_test2),
      shuffle=False
  )
  
  return model

In [ ]:
# учим sigmoid
train_model(make_sigmoid_model)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 6s 123us/sample - loss: 2.5027 - accuracy: 0.1059 - val_loss: 2.2598 - val_accuracy: 0.2147
Epoch 2/10
50000/50000 [==============================] - 5s 106us/sample - loss: 2.4087 - accuracy: 0.1292 - val_loss: 2.1442 - val_accuracy: 0.2567
Epoch 3/10
50000/50000 [==============================] - 5s 107us/sample - loss: 2.2895 - accuracy: 0.1797 - val_loss: 2.0614 - val_accuracy: 0.2785
Epoch 4/10
50000/50000 [==============================] - 5s 109us/sample - loss: 2.2228 - accuracy: 0.2034 - val_loss: 2.0201 - val_accuracy: 0.2975
Epoch 5/10
50000/50000 [==============================] - 5s 108us/sample - loss: 2.1688 - accuracy: 0.2202 - val_loss: 1.9881 - val_accuracy: 0.3071
Epoch 6/10
50000/50000 [==============================] - 5s 107us/sample - loss: 2.1195 - accuracy: 0.2369 - val_loss: 1.9585 - val_accuracy: 0.3192
Epoch 7/10
50000/50000 [==========================

In [ ]:
# учим sgd
def train_model(make_model_func=make_default_model, optimizer="sgd"):
  BATCH_SIZE = 512
  EPOCHS = 10

  K.clear_session()
  model = make_model_func()

  model.compile(
      loss='categorical_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy']
  )

  model.fit(
      x_train2, y_train2,  # нормализованные данные
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      validation_data=(x_test2, y_test2),
      shuffle=False
  )
  
  return model

In [ ]:
def make_bn_model():
    model = Sequential()
    model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(32, 32, 3)))
    model.add(L.BatchNormalization())
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.BatchNormalization())
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.BatchNormalization())
    model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
    model.add(L.BatchNormalization())
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Flatten())
    model.add(L.Dense(256, activation='relu'))
    model.add(L.BatchNormalization())
    model.add(Dropout(0.5))
    model.add(L.Dense(10, activation='softmax'))
    
    
    return model

In [ ]:
K.clear_session()
model = make_bn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9

In [ ]:
def train_model(make_model_func=make_bn_model, optimizer="adam"):
  BATCH_SIZE = 512
  EPOCHS = 30

  K.clear_session()
  model = make_model_func()

  model.compile(
      loss='categorical_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy']
  )

  model.fit(
      x_train2, y_train2,  # нормализованные данные
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      validation_data=(x_test2, y_test2),
      shuffle=False
  )
  
  return model

In [ ]:
# учим bn
train_model(make_bn_model)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
  512/50000 [..............................] - ETA: 39s

KeyboardInterrupt: ignored

In [ ]:
def make_sigmoid_bn_model():
    model = Sequential()
    model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(32, 32, 3)))
    model.add(L.BatchNormalization())
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.BatchNormalization())
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
    model.add(L.BatchNormalization())
    model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
    model.add(L.BatchNormalization())
    model.add(L.MaxPool2D())
    model.add(Dropout(0.25))
    model.add(L.Flatten())
    model.add(L.Dense(256, activation='sigmoid'))
    model.add(L.BatchNormalization())
    model.add(Dropout(0.5))
    model.add(L.Dense(10, activation='softmax'))
    
    
    return model
    

In [ ]:
K.clear_session()
model = make_sigmoid_bn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9

In [ ]:
def train_model(make_model_func=make_sigmoid_bn_model, optimizer="adam"):
  BATCH_SIZE = 512
  EPOCHS = 30

  K.clear_session()
  model = make_model_func()

  model.compile(
      loss='categorical_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy']
  )

  model.fit(
      x_train2, y_train2,  # нормализованные данные
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      validation_data=(x_test2, y_test2),
      shuffle=False
  )
  
  return model

In [ ]:
# учим sigmoid + bn
train_model(make_sigmoid_bn_model)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 8s 159us/sample - loss: 1.7451 - accuracy: 0.4272 - val_loss: 2.2970 - val_accuracy: 0.2008
Epoch 2/30
50000/50000 [==============================] - 7s 132us/sample - loss: 1.2725 - accuracy: 0.5549 - val_loss: 2.2688 - val_accuracy: 0.2684
Epoch 3/30
50000/50000 [==============================] - 7s 131us/sample - loss: 1.1309 - accuracy: 0.6034 - val_loss: 1.9406 - val_accuracy: 0.3453
Epoch 4/30
50000/50000 [==============================] - 6s 130us/sample - loss: 1.0503 - accuracy: 0.6304 - val_loss: 1.4939 - val_accuracy: 0.4992
Epoch 5/30
50000/50000 [==============================] - 7s 130us/sample - loss: 0.9923 - accuracy: 0.6563 - val_loss: 1.1274 - val_accuracy: 0.6174
Epoch 6/30
50000/50000 [==============================] - 7s 132us/sample - loss: 0.9343 - accuracy: 0.6739 - val_loss: 0.9646 - val_accuracy: 0.6673
Epoch 7/30
50000/50000 [==========================